In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from sklearn.metrics import *
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, train_test_split

train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/test.csv")
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')
submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sample_submission.csv')

data = train

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').sort_values('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['서울권여부'] = (feature['서울특별시여부'] + feature['경기도여부'] != 0) * 1
feature['광역시여부'] = (feature['대전광역시여부'] + feature['광주광역시여부'] + feature['부산광역시여부'] + feature['울산광역시여부'] + feature['대구광역시여부'] != 0) * 1
feature['시여부'] = (feature['서울권여부'] + feature['광역시여부'] + feature['세종특별자치시여부'] != 0) * 1

list3 = []
for i in train['공급유형'].unique():
  list3.append(i + '여부')
  feature[i + '여부'] = (feature['공급유형'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1

feature = pd.merge(feature, age, how = 'left', on = '지역')

feature['지하철존재여부'] = (feature['도보 10분거리 내 지하철역 수(환승노선 수 반영)'] != 0) * 1
feature['버스정류장존재여부'] = (feature['도보 10분거리 내 버스정류장 수'] != 0) * 1

feature['공가비율'] = feature['공가수'] / feature['총세대수']
feature['공가여부'] = (feature['공가수'] < 10) * 1
feature['공가수2'] = feature['공가수'] ** 2
feature['공가수3'] = np.sqrt(feature['공가수'])


feature['등록차량수'] = (feature['등록차량수'] - feature['단지내주차면수'])

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

for j in ['남자', '여자']:
  feature[j] = feature[age[0] + '(' + j + ')']
  for i in age[1:]:
    feature[j] += feature[i + '(' + j + ')']


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

In [ ]:
random_state_ = 0
n_estimators_ = 100

feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
s = []
for j in range(40):
  scores = []
  learning_rate_ = 0.01

  train_target = np.array(feature['등록차량수'])
  train_feature = np.array(feature['단지내주차면수'])
  train_X, test_X, train_y, test_y = train_test_split(train_feature, train_target, test_size = 0.1, shuffle = True, random_state=random_state_)
  b = test_X.copy()
  c = test_y

  for i in range(10):
    train_target = np.array(feature['등록차량수'] / feature['단지내주차면수'])
    train_feature = np.array(feature[feature_names_])
    train_X, test_X, train_y, test_y = train_test_split(train_feature, train_target, test_size = 0.1, shuffle = True, random_state=random_state_)
    rgr = LGBMRegressor(n_estimators=n_estimators_,
                        random_state=random_state_,
                        learning_rate=learning_rate_)
    
    rgr.fit(train_X, train_y)
    score = mean_absolute_error(test_y, rgr.predict(test_X))
    learning_rate_ += 0.01

    a = rgr.predict(test_X)
    sss = mean_absolute_error(c, b * a)
    scores.append(sss)
  print(min(scores))
  s.append(min(scores))
  random_state_ += 1
print("LGBM AUC: %0.4f (+/- %0.4f)" %(np.mean(s), np.std(s) * 2))

117.61441327353589
115.88207499400868
134.6971888856049
85.43352625711017
141.1792888570643
135.08513595371787
112.92357260796183
134.1268999745059
105.85091852208119
129.91295242780956
107.27406010492884
89.98886540342407
138.26086270939155
93.39397485146785
115.83498259865523
134.7643617927108
119.42764512893427
107.47079951302199
98.36880566495282
90.45669535041408
132.7289062925234
100.9515872412667
86.475914179886
93.16921876935784
106.49971345007131
124.61877768593618
104.19454127760572
100.29164333430388
132.02698998136944
111.76625373759241
109.17008182376573
127.98338516349145
117.57984545746002
121.73169829801398
133.01436006203977
88.41911105421637
105.98328868301802
107.53320483668696
120.19612723763545
120.33587374562923
LGBM AUC: 113.8154 (+/- 31.7752)


In [ ]:
1. split 1
2. grid search 100    0.01 ~ 0.1

4. random 0 ~ 40
3. 그래서 feature 골라냄
4. train 전체를 model
5. test

NameError: ignored

LGBM AUC: 117.7643
LGBM AUC: 117.4242
LGBM AUC: 122.3582
LGBM AUC: 124.9997
LGBM AUC: 125.7283
LGBM AUC: 124.6941
LGBM AUC: 126.0230
LGBM AUC: 125.4904
LGBM AUC: 123.6654
LGBM AUC: 124.2681
117.4242002888021
LGBM AUC: 115.7919
LGBM AUC: 118.7404
LGBM AUC: 119.5663
LGBM AUC: 122.0130
LGBM AUC: 123.0530
LGBM AUC: 125.1849
LGBM AUC: 125.7350
LGBM AUC: 128.4399
LGBM AUC: 129.2305
LGBM AUC: 130.2106
115.79185954582711
LGBM AUC: 134.6302
LGBM AUC: 139.7457
LGBM AUC: 142.7013
LGBM AUC: 145.0190
LGBM AUC: 145.5544
LGBM AUC: 149.5122
LGBM AUC: 147.9597
LGBM AUC: 148.0490
LGBM AUC: 148.6488
LGBM AUC: 150.0553
134.63016511706613
LGBM AUC: 108.4382
LGBM AUC: 94.3385
LGBM AUC: 88.0361
LGBM AUC: 87.6800
LGBM AUC: 88.0896
LGBM AUC: 87.0140
LGBM AUC: 86.4713
LGBM AUC: 86.2484
LGBM AUC: 87.2884
LGBM AUC: 86.8316
86.2484372001116
LGBM AUC: 151.5100
LGBM AUC: 145.3219
LGBM AUC: 141.9197
LGBM AUC: 143.1969
LGBM AUC: 143.1410
LGBM AUC: 143.1208
LGBM AUC: 145.7012
LGBM AUC: 148.5132
LGBM AUC: 148.5795
LGBM 

KeyboardInterrupt: ignored

In [ ]:
114.2072 (+/- 23.5809)
['평균전용면적', '공가수', '40대', '50대'] + list3
116.5968 (+/- 22.8330)

In [ ]:
['총세대수', '공가수', '단지내주차면수', '공가비율', 
                  '면적30', '면적40', '면적50', '평균전용면적', 
                  '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)']

['공가수', '단지내주차면수', '평균전용면적'] + list1 + list2

In [ ]:
train_target = np.array(feature['등록차량수']).astype(int)
feature_names_ = ['총세대수', '공가수', '단지내주차면수', '공가비율', 
                  '면적30', '면적40', '면적50', '평균전용면적', 
                  '40대', '50대', '10대', '10대미만',
                  '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)'] + list1
train_feature = np.array(feature[feature_names_])

random_state_ = 0
n_estimators_ = 100
learning_rate_ = 0.01

for k in range(20):
  s = []
  learning_rate_ = 0.01
  kf = KFold(n_splits = 10, shuffle = True, random_state = random_state_)
  for i in range(10):
    models = []
    scores = []
    for train_index, test_index in kf.split(train_feature):
      train_X, test_X, train_y, test_y = train_feature[train_index], train_feature[test_index], train_target[train_index], train_target[test_index]

      rgr = LGBMRegressor(n_estimators=n_estimators_,
                          random_state=random_state_,
                          learning_rate=learning_rate_)

      rgr.fit(train_X, train_y)
      score = mean_absolute_error(test_y, rgr.predict(test_X))
      scores.append(score)
    s.append(np.mean(scores))
    learning_rate_ += 0.01
  print("LGBM MAE: %0.4f" %(min(s)))
  random_state_ += 1

LGBM MAE: 116.6999
LGBM MAE: 115.7974
LGBM MAE: 116.1284
LGBM MAE: 120.8111
LGBM MAE: 121.3541
LGBM MAE: 119.0953
LGBM MAE: 119.4369
LGBM MAE: 116.1891
LGBM MAE: 116.2103
LGBM MAE: 117.4374
LGBM MAE: 117.2493
LGBM MAE: 116.3661
LGBM MAE: 117.8711
LGBM MAE: 120.0592
LGBM MAE: 116.9191
LGBM MAE: 118.5499
LGBM MAE: 117.3228
LGBM MAE: 115.7842
LGBM MAE: 115.3578
LGBM MAE: 117.1514


In [ ]:
train_target = np.array(feature['등록차량수']).astype(int)
feature_names_ = ['총세대수', '공가수', '단지내주차면수', '공가비율', 
                  '면적30', '면적40', '면적50', '평균전용면적', 
                  '40대(여자)', '40대(남자)', '50대(여자)', '50대(남자)']
train_feature = np.array(feature[feature_names_])

random_state_ = 0
n_estimators_ = 1000
learning_rate_ = 0.002

skf = KFold(n_splits = 10, shuffle = True, random_state = random_state_)

for i in range(10):
  models = []
  scores = []

  for train_index, test_index in skf.split(train_feature, train_target):
      train_X, test_X, train_y, test_y = train_feature[train_index], train_feature[test_index], train_target[train_index], train_target[test_index]

      rgr = LGBMRegressor(n_estimators=n_estimators_,
                          random_state=random_state_,
                          learning_rate=learning_rate_)
      
      rgr.fit(train_X, train_y)
      score = mean_absolute_error(test_y, rgr.predict(test_X))
      scores.append(score)
      models.append(rgr)

  print("LGBM AUC: %0.4f (+/- %0.4f)" %(np.mean(scores), np.std(scores) * 2))
  learning_rate_ += 0.001

LGBM AUC: 123.9646 (+/- 37.3240)


KeyboardInterrupt: ignored

In [ ]:
feature[feature_names_]

,단지내주차면수,공가비율,면적30,면적40,면적50,평균전용면적,40대(여자),40대(남자),50대(여자),50대(남자)
0,438.0,0.017668,0,419,147,42.175230,0.082318,0.072648,0.082747,0.074276
1,153.0,0.005758,3,506,1,39.490921,0.078860,0.073418,0.079299,0.069167
2,950.0,0.013986,0,0,904,53.441049,0.078860,0.073418,0.079299,0.069167
3,1119.0,0.012232,0,291,1017,45.952500,0.080028,0.079183,0.079902,0.067443
4,823.0,0.005020,516,0,396,41.475843,0.077005,0.066645,0.081408,0.074382
...,...,...,...,...,...,...,...,...,...,...
418,700.0,0.018135,112,220,440,41.832591,0.083208,0.078355,0.077345,0.064859
419,837.0,0.021218,0,458,626,43.768635,0.061952,0.060769,0.082899,0.068855
420,600.0,0.013889,192,312,144,40.569630,0.077005,0.066645,0.081408,0.074382
421,564.0,0.025066,0,486,272,40.247098,0.077092,0.067600,0.086873,0.072570


# CSV 파트

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

import numpy as np
import pandas as pd
from sklearn.metrics import *
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, train_test_split

train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/test.csv")
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')
submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sample_submission.csv')


data = train

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').sort_values('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1
# age = pd.read_csv('age_gender_info.csv')

feature = pd.merge(feature, age, how = 'left', on = '지역')

feature['공가비율'] = feature['공가수'] / feature['총세대수']

feature['등록차량수'] = (feature['등록차량수'] - feature['단지내주차면수'])

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

train_target = np.array(feature['등록차량수'] / feature['단지내주차면수'])#.astype(int)


feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
train_feature = np.array(feature[feature_names_])

data = test

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1
age = pd.read_csv('age_gender_info.csv')

feature = pd.merge(feature, age, how = 'left', on = '지역')

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
test_feature = np.array(feature[feature_names_])


random_state_ = 0
n_estimators_ = 1000
learning_rate_ = 0.003

train_X, train_y = train_feature, train_target

rgr = LGBMRegressor(n_estimators=n_estimators_,
                    random_state=random_state_,
                    learning_rate=learning_rate_)

rgr.fit(train_X, train_y)
tmp = feature['단지내주차면수']
prediction = rgr.predict(test_feature)

submission['num'] = np.round(tmp * prediction + tmp)
submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/submissionJunProbCheck.csv', index = False)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


FileNotFoundError: ignored

In [ ]:
submission

,code,num
0,C1072,749.0
1,C1128,1376.0
2,C1456,461.0
3,C1840,491.0
4,C1332,1309.0
...,...,...
145,C2456,278.0
146,C1266,420.0
147,C2152,31.0
148,C1267,323.0


In [ ]:
submission

,code,num
0,C1072,127.0
1,C1128,403.0
2,C1456,70.0
3,C1840,238.0
4,C1332,318.0
...,...,...
145,C2456,24.0
146,C1266,124.0
147,C2152,54.0
148,C1267,21.0


In [ ]:
import eli5 
from eli5.sklearn import PermutationImportance

random_state_ = 0
n_estimators_ = 100
learning_rate_ = 0.03

for i in range(20):


  train_X, test_X, train_y, test_y = train_test_split(train_feature, train_target, test_size = 0.3, shuffle = True, random_state=random_state_)

  rgr = LGBMRegressor(n_estimators=n_estimators_,
                      random_state=random_state_,
                      learning_rate=learning_rate_)

  rgr.fit(train_X, train_y)
  perm = PermutationImportance(rgr).fit(test_X, test_y) 
  display(eli5.explain_weights(perm, top = 100, feature_names = feature_names_))

  random_state_ += 1

Weight,Feature
0.1645 ± 0.1422,단지내주차면수
0.1423 ± 0.0334,평균전용면적
0.0642 ± 0.0227,면적30
0.0388 ± 0.0778,공가수
0.0263 ± 0.0155,총세대수
0.0227 ± 0.0410,총전용면적
0.0191 ± 0.0645,40대(남자)
0.0178 ± 0.0195,면적40
-0.0011 ± 0.0032,50대(남자)
-0.0019 ± 0.0087,40대(여자)


Weight,Feature
0.1061 ± 0.0475,단지내주차면수
0.0911 ± 0.0321,평균전용면적
0.0898 ± 0.0922,공가수
0.0607 ± 0.0392,면적40
0.0526 ± 0.0323,총전용면적
0.0431 ± 0.0370,면적30
0.0378 ± 0.0536,총세대수
0.0184 ± 0.0164,면적50
0.0183 ± 0.0157,40대(남자)
0.0096 ± 0.0054,40대(여자)


Weight,Feature
0.0891 ± 0.0334,50대(여자)
0.0486 ± 0.0185,40대(남자)
0.0475 ± 0.0329,총전용면적
0.0276 ± 0.0310,공가수
0.0122 ± 0.0114,면적50
0.0109 ± 0.0265,평균전용면적
0.0107 ± 0.0043,40대(여자)
-0.0008 ± 0.0077,50대(남자)
-0.0173 ± 0.1580,단지내주차면수
-0.0349 ± 0.0220,총세대수


Weight,Feature
0.1029 ± 0.0579,평균전용면적
0.0832 ± 0.0595,공가수
0.0377 ± 0.0361,면적40
0.0212 ± 0.0334,면적50
0.0203 ± 0.0047,단지내주차면수
0.0194 ± 0.0238,총전용면적
0.0138 ± 0.0137,50대(남자)
0.0131 ± 0.0108,40대(여자)
0.0114 ± 0.0083,40대(남자)
0.0100 ± 0.0049,50대(여자)


Weight,Feature
0.1418 ± 0.0191,단지내주차면수
0.1038 ± 0.0498,공가수
0.0534 ± 0.0532,40대(남자)
0.0488 ± 0.1268,면적40
0.0388 ± 0.0624,평균전용면적
0.0287 ± 0.0236,면적30
0.0137 ± 0.0143,40대(여자)
0.0026 ± 0.0074,50대(남자)
-0.0006 ± 0.0055,50대(여자)
-0.0051 ± 0.0361,면적50


...

Weight,Feature
0.1819 ± 0.1325,단지내주차면수
0.0836 ± 0.1055,공가수
0.0308 ± 0.1093,면적50
0.0285 ± 0.0601,총세대수
0.0195 ± 0.0514,40대(남자)
0.0162 ± 0.0341,면적30
0.0084 ± 0.0098,50대(여자)
0.0007 ± 0.0019,50대(남자)
0.0006 ± 0.0154,40대(여자)
-0.0085 ± 0.0445,면적40


Weight,Feature
0.1403 ± 0.0808,평균전용면적
0.1312 ± 0.0401,총전용면적
0.0469 ± 0.0318,공가수
0.0165 ± 0.0058,면적50
0.0081 ± 0.0122,40대(남자)
0.0075 ± 0.0083,50대(남자)
0.0047 ± 0.0638,면적40
0.0038 ± 0.0076,40대(여자)
-0.0085 ± 0.0067,50대(여자)
-0.0136 ± 0.0733,면적30


Weight,Feature
0.0902 ± 0.0520,공가수
0.0819 ± 0.0751,평균전용면적
0.0687 ± 0.0375,단지내주차면수
0.0213 ± 0.0294,면적50
0.0210 ± 0.0221,40대(남자)
0.0183 ± 0.0081,50대(남자)
0.0170 ± 0.0339,총전용면적
0.0056 ± 0.0549,면적40
0.0045 ± 0.0255,면적30
-0.0006 ± 0.0154,50대(여자)


Weight,Feature
0.0813 ± 0.0704,평균전용면적
0.0791 ± 0.0962,40대(남자)
0.0693 ± 0.1059,공가수
0.0686 ± 0.0377,면적50
0.0350 ± 0.0143,50대(여자)
0.0298 ± 0.0616,면적40
0.0253 ± 0.0704,단지내주차면수
0.0162 ± 0.0212,총전용면적
0.0029 ± 0.0072,50대(남자)
0.0016 ± 0.0029,40대(여자)


Weight,Feature
0.2041 ± 0.1178,단지내주차면수
0.0845 ± 0.0449,공가수
0.0801 ± 0.0888,40대(남자)
0.0509 ± 0.1063,총전용면적
0.0167 ± 0.0697,총세대수
0.0153 ± 0.0459,면적30
0.0010 ± 0.0080,50대(여자)
-0.0009 ± 0.0457,면적50
-0.0032 ± 0.0069,50대(남자)
-0.0040 ± 0.0086,40대(여자)
